# 99 Scala Exercises (31 to 41)

### 31\. Determine whether a given integer number is prime.

In [36]:
import scala.math.sqrt

implicit class IntOps(val x: Int) {
    // Simple recursive solution
    def isPrime(): Boolean = {
        val sqrtx = sqrt(x).toInt
        @annotation.tailrec
        def inner(i: Int): Boolean = {
            if (i > sqrtx)
                true
            else if (x % i == 0)
                false
            else
                inner(i+1)
        }
        inner(2)
    }
    
    // Concise solution using Streams
    def isPrimeLazy(): Boolean =
        !(Stream.range(2, sqrt(x).toInt).exists(x % _ == 0))
}

7.isPrime
120.isPrime
157.isPrime
157.isPrimeLazy
time { 550342.isPrime }
time { 550342.isPrimeLazy }

import scala.math.sqrt


defined class IntOps
res35_2: Boolean = true
res35_3: Boolean = false
res35_4: Boolean = true
res35_5: Boolean = true
res35_6: (Boolean, concurrent.duration.FiniteDuration) = (false, 27299 nanoseconds)
res35_7: (Boolean, concurrent.duration.FiniteDuration) = (false, 49209 nanoseconds)

### 32\. Determine the greatest common divisor of two positive integer numbers.
Use Euclid's algorithm.

In [20]:
@annotation.tailrec
def gcd(x: Int, y: Int): Int = {
    val rem = x % y
    if (rem == 0) y else gcd(y, rem)
}

gcd(36, 63)

defined function gcd
res19_1: Int = 9

### 33\. Determine whether two positive integer numbers are coprime.
Two numbers are coprime if their greatest common divisor equals 1.

In [23]:
implicit class IntOps2(val x: Int) {
    def isCoprimeTo(y: Int): Boolean = gcd(x, y) == 1
}

35 isCoprimeTo 64
32 isCoprimeTo 64

defined class IntOps2
res22_1: Boolean = true
res22_2: Boolean = false

### 34\. Calculate Euler's totient function phi(m).
Euler's so-called totient function phi(m) is defined as the number of positive integers r (1 <= r <= m) that are coprime to m.

In [51]:
implicit class IntOps3(val x: Int) {
    def totient(): Int = (1 to x).count(x isCoprimeTo _)
}

10 totient

defined class IntOps3
res50_1: Int = 4

### 35\. Determine the prime factors of a given positive integer.
Construct a flat list containing the prime factors in ascending order.

In [37]:
val primes: Stream[Int] = 2 #:: Stream.from(3).filter { i =>
   primes.takeWhile(p => p * p <= i).forall(p => i % p > 0) };

implicit class IntOps4(val x: Int) {
    def primeFactors(): List[Int] = {
        def inner(i: Int, s: Stream[Int]): List[Int] = s match{
            case _ if i isPrime => List(i)
            case h#::t if i % h == 0 =>  h :: inner(i/h, s)
            case _#::t => inner(i, t)
        }
        inner(x, primes)
    }
}
315.primeFactors

primes: Stream[Int] = Stream(
  2,
  3,
  5,
  7,
  11,
  13,
  17,
  19,
  23,
  29,
  31,
...
defined class IntOps4
res36_2: List[Int] = List(3, 3, 5, 7)

### 36\.Determine the prime factors of a given positive integer (2).
Construct a list containing the prime factors and their multiplicity.
Alternately, use a Map for the result.

In [44]:
import scala.collection.immutable.ListMap

implicit class IntOps5(val x: Int) {
    def primeFactorMultiplicity(): List[(Int, Int)] =
        x.primeFactors.groupBy(identity)
                      .mapValues { _.length }
                      .toList
                      .sortBy(_._1)
    
    def primeFactorMultiplicityMap(): Map[Int, Int] =
        ListMap(x.primeFactors
                    .groupBy(identity)
                    .mapValues { _.length }
                    .toSeq
                    .sortBy(_._1):_*)
}

315.primeFactorMultiplicity
315.primeFactorMultiplicityMap
time { 10324525.primeFactorMultiplicity }
time { 10324525.primeFactorMultiplicityMap }

import scala.collection.immutable.ListMap


defined class IntOps5
res43_2: List[(Int, Int)] = List((3, 2), (5, 1), (7, 1))
res43_3: Map[Int, Int] = Map(3 -> 2, 5 -> 1, 7 -> 1)
res43_4: (List[(Int, Int)], concurrent.duration.FiniteDuration) = (List((5, 2), (17, 2), (1429, 1)), 87298 nanoseconds)
res43_5: (Map[Int, Int], concurrent.duration.FiniteDuration) = (Map(5 -> 2, 17 -> 2, 1429 -> 1), 130726 nanoseconds)

### 37\. Calculate Euler's totient function phi(m) (improved).
See problem P34 for the definition of Euler's totient function. If the list of the prime factors of a number m is known in the form of problem P36 then the function phi(m>) can be efficiently calculated as follows: Let [[p1, m1], [p2, m2], [p3, m3], ...] be the list of prime factors (and their multiplicities) of a given number m. Then phi(m) can be calculated with the following formula:
    $$ phi(m) = (p1-1)*p1^{(m1-1)} * (p2-1)*p2^{(m2-1)} * (p3-1)*p3^{(m3-1)} * ... $$

In [57]:
implicit class IntOps6(val x: Int) {
    def totientImproved(): Int = x.primeFactorMultiplicity.foldLeft(1) {
       case (accum, (p, m)) => accum * (p-1) * Math.pow(p, m-1).toInt
    }
}

time { 10000000.totientImproved }
time { 10000000.totient }

defined class IntOps6
res56_1: (Int, concurrent.duration.FiniteDuration) = (4000000, 521831 nanoseconds)
res56_2: (Int, concurrent.duration.FiniteDuration) = (4000000, 1335419922 nanoseconds)

### 38\. Compare the two methods of calculating Euler's totient function.
Use the solutions of problems P34 and P37 to compare the algorithms. Try to calculate phi(10090) as an example.

In [61]:
(for (_ <- 1 to 10000) yield time(10090 totientImproved)._2.toNanos).sum / 10000
(for (_ <- 1 to 10000) yield time(10090 totient)._2.toNanos).sum / 10000

res60_0: Long = 4257L
res60_1: Long = 889187L

### 39\. A list of prime numbers.
Given a range of integers by its lower and upper limit, construct a list of all prime numbers in that range.

In [63]:
def listPrimesInRange(range: Seq[Int]): List[Int] =
   primes.dropWhile(_ < range.head).takeWhile(_ < range.last).toList

listPrimesInRange(7 to 31)

defined function listPrimesInRange
res62_1: List[Int] = List(7, 11, 13, 17, 19, 23, 29)

### 40\. Goldbach's conjecture.
Goldbach's conjecture says that every positive even number greater than 2 is the sum of two prime numbers. E.g. 28 = 5 + 23. It is one of the most famous facts in number theory that has not been proved to be correct in the general case. It has been numerically confirmed up to very large numbers (much larger than Scala's Int can represent). Write a function to find the two prime numbers that sum up to a given even integer.

In [72]:
implicit class IntOps7(val x: Int) {
    def goldbach(): Option[(Int, Int)] = {
        val p = primes.find(p => (x - p).isPrime)
        p map { y => (y, x - y) }
    }
}

28.goldbach

defined class IntOps7
res71_1: Option[(Int, Int)] = Some((5, 23))

### 41\. A list of Goldbach compositions.
Given a range of integers by its lower and upper limit, print a list of all even numbers and their Goldbach composition.

In [84]:
import scalaz.effect.IO._

def printGoldbachList(range: Seq[Int]): IO[Unit] = (for {
    n <- range if n % 2 == 0
    io = (n goldbach).fold(ioUnit) { case (p1, p2) => putStrLn(s"$n = $p1 + $p2") }
} yield io).foldLeft(ioUnit)(_ |+| _)

printGoldbachList(9 to 20).unsafePerformIO

10 = 3 + 7
12 = 5 + 7
14 = 3 + 11
16 = 3 + 13
18 = 5 + 13
20 = 3 + 17


import scalaz.effect.IO._


defined function printGoldbachList

In most cases, if an even number is written as the sum of two prime numbers, one of them is very small. Very rarely, the primes are both bigger than, say, 50. Try to find out how many such cases there are in the range 2..3000.

In [87]:
def printGoldbachListLimited(range: Seq[Int], limit: Int): IO[Unit] = (for {
    n <- range if n % 2 == 0
    io = (n goldbach).fold(ioUnit) {
        case (p1, p2) if p1 > limit && p2 > limit => putStrLn(s"$n = $p1 + $p2")
        case _ => ioUnit
    }
} yield io).foldLeft(ioUnit)(_ |+| _)

printGoldbachListLimited(1 to 2000, 50).unsafePerformIO

992 = 73 + 919
1382 = 61 + 1321
1856 = 67 + 1789
1928 = 61 + 1867


defined function printGoldbachListLimited